In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!source /content/drive/MyDrive/colab_env/bin/activate

In [ ]:
! pip install -q geemap wxee

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00


In [14]:
import xarray as xr
import ee
import itertools
import pickle
import pandas as pd
import wxee
import rasterio
import numpy as np
from datetime import datetime
import geopandas as gpd
from shapely.geometry import mapping
import geemap
from tqdm import tqdm

In [15]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file="/content/drive/MyDrive/private-key.json"
)

ee.Initialize(credentials)

In [ ]:

# /content/drive/MyDrive/EC_Tower/shapefile/central_valley_3857.shp
shapefile_basins = gpd.read_file("C:\Users\acer\Desktop\final_code\data\shape_file_region\central_valley_3857.shp")

shapefile_basins = shapefile_basins.to_crs("epsg:4326")
shapefile_basins.crs = {'init': 'epsg:4326'}

fc_shapefile_basins = geemap.geopandas_to_ee(shapefile_basins)
fc_shapefile_basins


/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [17]:
# Define the TerraClimate variables you are interested in
variables = [
    'aet', 'def', 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad',
    'swe', 'tmmx', 'tmmn', 'vap', 'vpd', 'vs'
]
start_date = '1993-01-01'  # Replace with your start date
end_date = '2022-12-31'  # Replace with your end date



In [18]:

# Step 3: Filter the dataset by date and region

terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')

filtered_data = terraclimate.filterBounds(fc_shapefile_basins).filterDate(start_date, end_date)

# Function to extract values for each pixel
def extract_values(image):
    values = image.select(variables).addBands(
        ee.Image.pixelLonLat()).reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=fc_shapefile_basins,
            scale=4638.3,  # Terra Climate resolution
            maxPixels=1e8
        )
    return ee.Feature(None, values).set('date', image.date().format('YYYY-MM-dd'))

# Extract values
data_list = filtered_data.map(extract_values).getInfo()

In [19]:
# Prepare data for DataFrame
rows = []
for feature in data_list['features']:
    props = feature['properties']
    lats = props['latitude']
    lons = props['longitude']
    date = props['date']
    for i in range(len(lats)):
        row = {
            'latitude': lats[i],
            'longitude': lons[i],
            'date': date
        }
        for var in variables:
            row[var] = props[var][i]
        rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)
df

,latitude,longitude,date,aet,def,pdsi,pet,pr,ro,soil,srad,swe,tmmx,tmmn,vap,vpd,vs
0,40.645962,-122.270984,1993-01-01,341,0,116,345,372,337,3476,690,0,105,7,607,35,249
1,40.604296,-122.395984,1993-01-01,349,0,116,353,350,300,3536,690,0,108,11,613,37,244
2,40.604296,-122.354317,1993-01-01,345,0,118,349,353,299,3520,690,0,106,10,613,36,249
3,40.604296,-122.312651,1993-01-01,339,0,121,343,350,293,3504,690,0,105,7,613,35,249
4,40.604296,-122.270984,1993-01-01,331,0,123,335,351,299,3488,690,0,103,6,614,34,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097995,34.979310,-118.854326,2022-12-01,421,0,-237,421,51,3,113,826,0,157,34,853,43,220
1097996,34.979310,-118.812660,2022-12-01,442,0,-244,442,52,3,99,827,0,157,34,829,46,230
1097997,34.937644,-118.979326,2022-12-01,446,0,-204,446,79,4,359,850,0,146,32,771,45,240
1097998,34.937644,-118.937660,2022-12-01,438,0,-207,438,73,4,307,847,0,150,32,793,45,230


In [20]:
# sclaling
df["aet"] = 0.1 * df["aet"]
df["def"] = 0.1 * df["def"]
df["pdsi"] = 0.01 * df["pdsi"]
df["pet"] = 0.1 * df["pet"]
df["soil"] = 0.1 * df["soil"]
df["srad"] = 0.1 * df["srad"]
df["tmmx"] = 0.1 * df["tmmx"]
df["tmmn"] = 0.1 * df["tmmn"]
df["vap"] = 0.001 * df["vap"]
df["vpd"] = 0.01 * df["vpd"]
df["vs"] = 0.01 * df["vs"]
df["delta_s"] = df["pr"] - df["ro"] - df["aet"]

df['time'] = pd.to_datetime(df['date'],format= "%Y-%m-%d" )

df["Month"] = df["time"].apply(lambda x: x.month)
df["Year"] = df["time"].apply(lambda x: x.year)
df

,latitude,longitude,date,aet,def,pdsi,pet,pr,ro,soil,...,swe,tmmx,tmmn,vap,vpd,vs,delta_s,time,Month,Year
0,40.645962,-122.270984,1993-01-01,34.1,0.0,1.16,34.5,372,337,347.6,...,0,10.5,0.7,0.607,0.35,2.49,0.9,1993-01-01,1,1993
1,40.604296,-122.395984,1993-01-01,34.9,0.0,1.16,35.3,350,300,353.6,...,0,10.8,1.1,0.613,0.37,2.44,15.1,1993-01-01,1,1993
2,40.604296,-122.354317,1993-01-01,34.5,0.0,1.18,34.9,353,299,352.0,...,0,10.6,1.0,0.613,0.36,2.49,19.5,1993-01-01,1,1993
3,40.604296,-122.312651,1993-01-01,33.9,0.0,1.21,34.3,350,293,350.4,...,0,10.5,0.7,0.613,0.35,2.49,23.1,1993-01-01,1,1993
4,40.604296,-122.270984,1993-01-01,33.1,0.0,1.23,33.5,351,299,348.8,...,0,10.3,0.6,0.614,0.34,2.50,18.9,1993-01-01,1,1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097995,34.979310,-118.854326,2022-12-01,42.1,0.0,-2.37,42.1,51,3,11.3,...,0,15.7,3.4,0.853,0.43,2.20,5.9,2022-12-01,12,2022
1097996,34.979310,-118.812660,2022-12-01,44.2,0.0,-2.44,44.2,52,3,9.9,...,0,15.7,3.4,0.829,0.46,2.30,4.8,2022-12-01,12,2022
1097997,34.937644,-118.979326,2022-12-01,44.6,0.0,-2.04,44.6,79,4,35.9,...,0,14.6,3.2,0.771,0.45,2.40,30.4,2022-12-01,12,2022
1097998,34.937644,-118.937660,2022-12-01,43.8,0.0,-2.07,43.8,73,4,30.7,...,0,15.0,3.2,0.793,0.45,2.30,25.2,2022-12-01,12,2022


In [ ]:
monthly_data = df.groupby(['Year', 'Month']).agg({'aet' : 'median',
                                                  'def' : 'median',
                                                  'pr' : 'median',
                                                  'pet' : 'median',
                                                  'delta_s' : 'median'}).reset_index()

monthly_data

,Year,Month,aet,def,pr,pet,delta_s
0,1993,1,3.390,0.000,129.0,3.40,84.190
1,1993,2,5.150,0.000,84.0,5.16,48.630
2,1993,3,7.210,1.740,47.0,8.98,37.965
3,1993,4,6.935,6.280,12.0,12.89,5.005
4,1993,5,3.500,13.425,12.0,17.00,7.085
...,...,...,...,...,...,...,...
355,2022,8,0.220,21.965,0.0,22.31,-0.140
356,2022,9,2.050,14.695,20.0,16.95,16.930
357,2022,10,0.130,11.300,0.0,11.47,-0.110
358,2022,11,3.410,2.090,35.0,5.75,29.490


In [ ]:
# /content/drive/MyDrive/EC_Tower/result/terra_climate_monthly_data.csv
monthly_data.to_csv('C:\Users\acer\Desktop\final_code\export\terra_data\data_monthly_basin.csv', index=False)

In [ ]:
monthly_data["pr_delta_s"] = monthly_data["pr"] - monthly_data["delta_s"]

monthly_data["aet/pr_delta_s"] = monthly_data["aet"] / monthly_data["pr_delta_s"]
monthly_data["pet/pr_delta_s"] = monthly_data["pet"] / monthly_data["pr_delta_s"]

monthly_data["aet/pr"] = monthly_data["aet"] / monthly_data["pr"]
monthly_data["pet/pr"] = monthly_data["pet"] / monthly_data["pr"]
monthly_data

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Year,Month,aet,def,pr,pet,delta_s,pr_delta_s,aet/pr_delta_s,pet/pr_delta_s,aet/pr,pet/pr
0,1993,1,33.90,0.00,129.0,34.0,54.10,74.90,0.452603,0.453939,0.262791,0.263566
1,1993,2,51.50,0.00,84.0,51.6,0.50,83.50,0.616766,0.617964,0.613095,0.614286
2,1993,3,72.10,17.40,47.0,89.8,-23.90,70.90,1.016925,1.266573,1.534043,1.910638
3,1993,4,69.35,62.80,12.0,128.9,-45.90,57.90,1.197755,2.226252,5.779167,10.741667
4,1993,5,35.00,134.25,12.0,170.0,-19.30,31.30,1.118211,5.431310,2.916667,14.166667
...,...,...,...,...,...,...,...,...,...,...,...,...
355,2022,8,2.20,219.65,0.0,223.1,-1.80,1.80,1.222222,123.944444,inf,inf
356,2022,9,20.50,146.95,20.0,169.5,-1.50,21.50,0.953488,7.883721,1.025000,8.475000
357,2022,10,1.30,113.00,0.0,114.7,-1.20,1.20,1.083333,95.583333,inf,inf
358,2022,11,34.10,20.90,35.0,57.5,-0.70,35.70,0.955182,1.610644,0.974286,1.642857


In [ ]:
yearly_data = df.groupby(['Year']).agg({'aet' : 'sum',
                                         'def' : 'sum',
                                         'pr' : 'sum',
                                         'pet' : 'sum',
                                        'delta_s' : 'sum'}).reset_index()

yearly_data

,Year,aet,def,pr,pet,delta_s
0,1993,1173302.4,3301896.8,1298714,4475349.9,-146072.4
1,1994,767369.7,3733090.0,917858,4500601.0,101091.3
2,1995,1160233.9,3232935.4,1594972,4393188.3,42822.1
3,1996,1228934.5,3441835.7,1540785,4670755.3,91823.5
4,1997,932970.4,3645385.2,1015935,4578486.8,-97541.4
5,1998,1300468.0,2792649.6,1666869,4094004.8,-82006.0
6,1999,808905.6,3598977.6,799103,4407901.8,-84771.6
7,2000,1060400.1,3408641.1,1176850,4469039.6,-14036.1
8,2001,923418.1,3637519.3,1292383,4561027.3,266749.9
9,2002,811434.2,3693651.5,924691,4505245.8,2911.8


In [ ]:
yearly_data["pr_delta_s"] = yearly_data["pr"] - yearly_data["delta_s"]

yearly_data["aet/pr_delta_s"] = yearly_data["aet"] / yearly_data["pr_delta_s"]
yearly_data["pet/pr_delta_s"] = yearly_data["pet"] / yearly_data["pr_delta_s"]

yearly_data["aet/pr"] = yearly_data["aet"] / yearly_data["pr"]
yearly_data["pet/pr"] = yearly_data["pet"] / yearly_data["pr"]
yearly_data

,Year,aet,def,pr,pet,delta_s,pr_delta_s,aet/pr_delta_s,pet/pr_delta_s,aet/pr,pet/pr
0,1993,1173302.4,3301896.8,1298714,4475349.9,-146072.4,1444786.4,0.812094,3.097586,0.903434,3.445986
1,1994,767369.7,3733090.0,917858,4500601.0,101091.3,816766.7,0.939521,5.510265,0.836044,4.903374
2,1995,1160233.9,3232935.4,1594972,4393188.3,42822.1,1552149.9,0.747501,2.830389,0.727432,2.754398
3,1996,1228934.5,3441835.7,1540785,4670755.3,91823.5,1448961.5,0.848148,3.223519,0.797603,3.031413
4,1997,932970.4,3645385.2,1015935,4578486.8,-97541.4,1113476.4,0.837890,4.111885,0.918337,4.506673
5,1998,1300468.0,2792649.6,1666869,4094004.8,-82006.0,1748875.0,0.743603,2.340936,0.780186,2.456105
6,1999,808905.6,3598977.6,799103,4407901.8,-84771.6,883874.6,0.915181,4.987022,1.012267,5.516062
7,2000,1060400.1,3408641.1,1176850,4469039.6,-14036.1,1190886.1,0.890429,3.752701,0.901049,3.797459
8,2001,923418.1,3637519.3,1292383,4561027.3,266749.9,1025633.1,0.900340,4.447036,0.714508,3.529161
9,2002,811434.2,3693651.5,924691,4505245.8,2911.8,921779.2,0.880291,4.887554,0.877519,4.872164


In [ ]:
yearly_data.to_csv('C:\Users\acer\Desktop\final_code\export\terra_data\data_basin.csv', index=False)
# /content/drive/MyDrive/EC_Tower/result/terra_climate_yearly_data.csv